In [29]:
from Function import cargar_dataset


In [45]:
from Function import importar_librerias


In [23]:
file_path = '/Users/Usuario/Desktop/prueba_citec/data_set/OnlineRetail.xlsx'
# Cargar el dataset
df = cargar_dataset(file_path)

1. Cleaned data

In [48]:
# Cambiemos 'CustomerID' a string
# Basados en las cloncluciones del  analilsis exploratorion del dataset procedo de la siguiente forma:
df['CustomerID'] = df['CustomerID'].fillna(1)
# Primero, rellené los nan o null  con 1 donde este pasa a ser los clientes no registrados luego convertí a int para eliminar los .0 en cada número, y finalmente a string


In [51]:
# Convertir a strings
df['CustomerID'] = df['CustomerID'].astype(str)

In [52]:
#se valida la totalidad de datos nulods, Quedaron sólo en 'Description', lo que no es importante para el análisis
df.isnull().sum()

InvoiceNo         0
StockCode         0
Description    1454
Quantity          0
InvoiceDate       0
UnitPrice         0
CustomerID        0
Country           0
dtype: int64

In [53]:
df.dtypes


InvoiceNo            category
StockCode            category
Description          category
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID             object
Country              category
dtype: object

In [54]:
# Eliminamos duplicados
df = df.drop_duplicates()

In [56]:
# el numero de registro encontrado en el analisis exploratorio fue de 541909 , por  lo tanto se encontraron 5268 datos duplicados 
df.shape 

(536641, 8)

In [57]:
df[df['Description'].isnull()]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
622,536414,22139,NaN,56,2010-12-01 11:52:00,0.0,Unknown,United Kingdom
1970,536545,21134,NaN,1,2010-12-01 14:32:00,0.0,Unknown,United Kingdom
1971,536546,22145,NaN,1,2010-12-01 14:33:00,0.0,Unknown,United Kingdom
1972,536547,37509,NaN,1,2010-12-01 14:33:00,0.0,Unknown,United Kingdom
1987,536549,85226A,NaN,1,2010-12-01 14:34:00,0.0,Unknown,United Kingdom
...,...,...,...,...,...,...,...,...
535322,581199,84581,NaN,-2,2011-12-07 18:26:00,0.0,Unknown,United Kingdom
535326,581203,23406,NaN,15,2011-12-07 18:31:00,0.0,Unknown,United Kingdom
535332,581209,21620,NaN,6,2011-12-07 18:35:00,0.0,Unknown,United Kingdom
536981,581234,72817,NaN,27,2011-12-08 10:33:00,0.0,Unknown,United Kingdom
